In [24]:
# http://localhost:8888/tree?token=828ad94db4d7d69d99c8c59436853dc031be47d05fee9a61

In [1]:
# ! gcloud auth login

In [2]:
# SERVICE_ACCOUNT = "flight-ml-demo-general@aia-ds-accelerator-flight-1.iam.gserviceaccount.com"  # @param {type:"string"}

In [8]:
from deltalake import DeltaTable
import pandas as pd
import gcsfs
import os
from utils import JSON_EncoderDecoder
import json
import os


pd.set_option("display.max_columns", 100)
pd.set_option("display.max_columns", 100)
pd.set_option('display.width', 4000)


def get_gcp_creds_json():
    gcp_creds_encoded = os.environ.get("GCP_CREDENTIALS_JSON_ENCODED")
    gcp_creds_json = JSON_EncoderDecoder(gcp_creds_encoded).decode().get()
    return gcp_creds_json


def init_storage_client():
    from google.oauth2 import service_account
    from google.cloud import storage
    gcp_creds_json = get_gcp_creds_json()
    
    gcp_credentials = service_account.Credentials.from_service_account_info(gcp_creds_json)
    storage_client = storage.Client(credentials=gcp_credentials)
    return storage_client


def get_date_from_lookback(lookback_days: int):
    from datetime import datetime, timedelta
    # Calculate the target date by subtracting the lookback_days from the current UTC date
    target_date = datetime.utcnow() - timedelta(days=lookback_days)
    # Extract year, month, and day
    year, month, day = target_date.year, target_date.month, target_date.day
    return f'{year}/{month:02d}/{day:02d}'


def read_data_preprocessed(table_path='gs://datalake-flight-dev-1/flightsummary-delta-processed-stream'):

    gcp_creds_json_str = json.dumps(get_gcp_creds_json())

    storage_options = {'service_account_key': gcp_creds_json_str}

    dt = DeltaTable(table_path, storage_options=storage_options)
    
    start_date = get_date_from_lookback(lookback_days=365)
    print(f"Reading data from {start_date}")

    df = dt.to_pandas(partitions=[("crt_ts_date", ">=", start_date)])
    return df


def create_target(df):
    """
    Create a target variable for modelling. 
    The target is the number of minutes between the actual landing time and the event time.
    """

    # group by flight id to get all the landing times
    landing_times = df[['fa_flight_id','actual_in']].groupby('fa_flight_id').max()['actual_in']

    # fill in the landing times 
    df['actual_in_filled'] = df['fa_flight_id'].map(landing_times)
    
    # calculate target in minutes
    df['target'] = (df['actual_in_filled'] - df['event_ts']).dt.total_seconds() / 60
    
    # Color the target so we can see it better
    df.style.bar(subset=['target'], color='#d65f5f')
    
    # Sort based on the time which events are SUPPOSED to happened
    df = df.sort_values(by=['actual_in_filled','crt_ts'])
    return df

def remove_incomplete_flights(df):
    " Remove flights which don't have a landing time"
    return df[df['actual_in_filled'].notnull()]

def removed_arrival_events(df):
    " For modelling, we only want to look at events which happen before landing"
    return df[df['event_type'] != 'actual_in']

# def style_datframe(df):
#     def highlight_scheduled_out(row):
#         if row['event_type'] == 'scheduled_out':
#             return ['background-color: lightgreen; color: black'] * len(row)
#         return [''] * len(row)

#     # Apply the styling
#     return df.style.apply(highlight_scheduled_out, axis=1)

def write_data_to_gcs(df, path_out: str):
    client = init_storage_client()
    bucket = client.get_bucket('datalake-flight-dev-1')
    blob = bucket.blob(f'{path_out}.csv')
    blob.upload_from_string(df.to_csv(), 'text/csv')


def create_training_data(path_out: str):
    df = read_data_preprocessed()
    df = create_target(df)
    df = remove_incomplete_flights(df)
    df = removed_arrival_events(df)
    write_data_to_gcs(df, path_out = path_out)
    return df


# create_training_data().count()

In [9]:
create_training_data(path_out='training/flights-modelready1')

Reading data from 2022/08/24


,ident,ident_icao,ident_iata,fa_flight_id,operator,operator_icao,operator_iata,flight_number,registration,atc_ident,inbound_fa_flight_id,codeshares,codeshares_iata,blocked,diverted,cancelled,position_only,departure_delay,arrival_delay,filed_ete,foresight_predictions_available,scheduled_out,estimated_out,actual_out,scheduled_off,estimated_off,actual_off,scheduled_on,estimated_on,actual_on,scheduled_in,estimated_in,actual_in,progress_percent,status,aircraft_type,route_distance,filed_airspeed,filed_altitude,route,baggage_claim,seats_cabin_business,seats_cabin_coach,seats_cabin_first,gate_origin,gate_destination,terminal_origin,terminal_destination,type,origin_code,origin_code_icao,origin_code_iata,origin_code_lid,origin_timezone,origin_name,origin_city,origin_airport_info_url,destination_code,destination_code_icao,destination_code_iata,destination_code_lid,destination_timezone,destination_name,destination_city,destination_airport_info_url,crt_ts,crt_ts_year,crt_ts_month,crt_ts_day,crt_ts_hour,last_run_ts,last_scheduled_out_ts,crt_ts_date,ingest_ts,event_type,event_ts,actual_in_filled,target
53,AAL2563,AAL2563,AA2563,AAL2563-1691323529-airline-902p,AAL,AAL,AA,2563,N9016,None,AAL1586-1691257539-airline-213p,['JAL7235'],['JL7235'],False,False,False,False,-360,180,2520,True,2023-08-08 12:00:00,2023-08-08 11:54:00,2023-08-08 11:54:00,2023-08-08 12:10:00,2023-08-08 12:16:34,2023-08-08 12:16:34,2023-08-08 12:52:00,2023-08-08 12:56:18,2023-08-08 12:56:18,2023-08-08 13:12:00,2023-08-08 13:07:00,2023-08-08 13:15:00,100,Arrived / Gate Arrival,A319,191,482,230.0,ELOEL3 FORSS GUTZZ BOOVE6,B5,None,None,None,22,B12,None,B,Airline,KAUS,KAUS,AUS,AUS,America/Chicago,Austin-Bergstrom Intl,Austin,/airports/KAUS,KDFW,KDFW,DFW,DFW,America/Chicago,Dallas-Fort Worth Intl,Dallas-Fort Worth,/airports/KDFW,2023-08-14 18:50:01,2023,8,14,18,2000-01-01 00:00:00,None,2023-08-14,2023-08-24 17:20:30.930,scheduled_out,2023-08-08 12:00:00,2023-08-08 13:15:00,75.000000
54,AAL2563,AAL2563,AA2563,AAL2563-1691323529-airline-902p,AAL,AAL,AA,2563,N9016,None,AAL1586-1691257539-airline-213p,['JAL7235'],['JL7235'],False,False,False,False,-360,180,2520,True,2023-08-08 12:00:00,2023-08-08 11:54:00,2023-08-08 11:54:00,2023-08-08 12:10:00,2023-08-08 12:16:34,2023-08-08 12:16:34,2023-08-08 12:52:00,2023-08-08 12:56:18,2023-08-08 12:56:18,2023-08-08 13:12:00,2023-08-08 13:07:00,2023-08-08 13:15:00,100,Arrived / Gate Arrival,A319,191,482,230.0,ELOEL3 FORSS GUTZZ BOOVE6,B5,None,None,None,22,B12,None,B,Airline,KAUS,KAUS,AUS,AUS,America/Chicago,Austin-Bergstrom Intl,Austin,/airports/KAUS,KDFW,KDFW,DFW,DFW,America/Chicago,Dallas-Fort Worth Intl,Dallas-Fort Worth,/airports/KDFW,2023-08-14 18:50:01,2023,8,14,18,2000-01-01 00:00:00,None,2023-08-14,2023-08-24 17:20:30.930,actual_out,2023-08-08 11:54:00,2023-08-08 13:15:00,81.000000
55,AAL2563,AAL2563,AA2563,AAL2563-1691323529-airline-902p,AAL,AAL,AA,2563,N9016,None,AAL1586-1691257539-airline-213p,['JAL7235'],['JL7235'],False,False,False,False,-360,180,2520,True,2023-08-08 12:00:00,2023-08-08 11:54:00,2023-08-08 11:54:00,2023-08-08 12:10:00,2023-08-08 12:16:34,2023-08-08 12:16:34,2023-08-08 12:52:00,2023-08-08 12:56:18,2023-08-08 12:56:18,2023-08-08 13:12:00,2023-08-08 13:07:00,2023-08-08 13:15:00,100,Arrived / Gate Arrival,A319,191,482,230.0,ELOEL3 FORSS GUTZZ BOOVE6,B5,None,None,None,22,B12,None,B,Airline,KAUS,KAUS,AUS,AUS,America/Chicago,Austin-Bergstrom Intl,Austin,/airports/KAUS,KDFW,KDFW,DFW,DFW,America/Chicago,Dallas-Fort Worth Intl,Dallas-Fort Worth,/airports/KDFW,2023-08-14 18:50:01,2023,8,14,18,2000-01-01 00:00:00,None,2023-08-14,2023-08-24 17:20:30.930,actual_off,2023-08-08 12:16:34,2023-08-08 13:15:00,58.433333
56,AAL2563,AAL2563,AA2563,AAL2563-1691323529-airline-902p,AAL,AAL,AA,2563,N9016,None,AAL1586-1691257539-airline-213p,['JAL7235'],['JL7235'],False,False,False,False,-360,180,2520,True,2023-08-08 12:00:00,2023-08-08 11:54:00,2023-08-08 11:54:00,2023-08-08 12:10:00,2023-08-08 12:16:34,2023-08-08 12:16:34,2023-08-08 12:52:0